In [0]:
spark.sql("use globalretail_silver")
spark.sql("""
CREATE TABLE IF NOT EXISTS silver_products (
    product_id STRING,
    name STRING,
    category STRING,
    brand STRING,
    price DOUBLE,
    stock_quantity INT,
    rating DOUBLE,
    is_active BOOLEAN,
    price_category STRING,
    stock_status STRING,
    last_updated TIMESTAMP
)
USING DELTA
""")

DataFrame[]

In [0]:
last_processed_df = spark.sql("SELECT MAX(last_updated) as last_processed FROM silver_products")
last_processed_timestamp = last_processed_df.collect()[0]['last_processed']

if last_processed_timestamp is None:
    last_processed_timestamp = "1900-01-01T00:00:00.000+00:00"

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMPORARY VIEW bronze_incremental_products AS
SELECT *
FROM globalretail_bronze.bronze_products WHERE ingestion_timestamp > '{last_processed_timestamp}'

""")

DataFrame[]

Data Transformations:
   - Price normalization (setting negative prices to 0)
   - Stock quantity normalization (setting negative stock to 0)
   - Rating normalization (clamping between 0 and 5)
   - Price categorization (Premium, Standard, Budget)
   - Stock status calculation (Out of Stock, Low Stock, Moderate Stock, Sufficient Stock)


In [0]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW silver_incremental_products AS
SELECT
    product_id,
    name,
    category,
    brand,
    CASE
        WHEN price < 0 THEN 0
        ELSE price
    END AS price,
    CASE
        WHEN stock_quantity < 0 THEN 0
        ELSE stock_quantity
    END AS stock_quantity,
    CASE
        WHEN rating < 0 THEN 0
        WHEN rating > 5 THEN 5
        ELSE rating
    END AS rating,
    is_active,
    CASE
        WHEN price > 1000 THEN 'Premium'
        WHEN price > 100 THEN 'Standard'
        ELSE 'Budget'
    END AS price_category,
    CASE
        WHEN stock_quantity = 0 THEN 'Out of Stock'
        WHEN stock_quantity < 10 THEN 'Low Stock'
        WHEN stock_quantity < 50 THEN 'Moderate Stock'
        ELSE 'Sufficient Stock'
    END AS stock_status,
    CURRENT_TIMESTAMP() AS last_updated
FROM bronze_incremental_products
WHERE name IS NOT NULL AND category IS NOT NULL
""")

DataFrame[]

In [0]:
%sql
select * from silver_incremental_products

product_id,name,category,brand,price,stock_quantity,rating,is_active,price_category,stock_status,last_updated
1,Product 1,Toys,BeautyGlow,995.73,989,3.5,true,Standard,Sufficient Stock,2024-07-19T05:28:38.635Z
2,Product 2,Garden,GardenMaster,497.76,495,3.8,true,Standard,Sufficient Stock,2024-07-19T05:28:38.635Z
3,Product 3,Electronics,BeautyGlow,331.63,10,4.6,true,Standard,Moderate Stock,2024-07-19T05:28:38.635Z
4,Product 4,Beauty,TechPro,798.83,683,4.7,false,Standard,Sufficient Stock,2024-07-19T05:28:38.635Z
5,Product 5,Automotive,HomeSmart,0.0,719,4.4,false,Budget,Sufficient Stock,2024-07-19T05:28:38.635Z
6,Product 6,Electronics,BookWorm,645.3,823,2.2,false,Standard,Sufficient Stock,2024-07-19T05:28:38.635Z
7,Product 7,Automotive,FashionX,549.08,999,1.1,false,Standard,Sufficient Stock,2024-07-19T05:28:38.635Z
8,Product 8,Books,TechPro,982.36,542,2.4,false,Standard,Sufficient Stock,2024-07-19T05:28:38.635Z
9,Product 9,Toys,FashionX,307.14,671,1.0,true,Standard,Sufficient Stock,2024-07-19T05:28:38.635Z
10,Product 10,Garden,BeautyGlow,871.38,975,3.4,false,Standard,Sufficient Stock,2024-07-19T05:28:38.635Z


In [0]:
spark.sql("""
MERGE INTO silver_products target
USING silver_incremental_products source
ON target.product_id = source.product_id
WHEN MATCHED THEN
    UPDATE SET *
WHEN NOT MATCHED THEN
    INSERT *
""")

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select * from  silver_products

product_id,name,category,brand,price,stock_quantity,rating,is_active,price_category,stock_status,last_updated
1,Product 1,Toys,BeautyGlow,995.73,989,3.5,true,Standard,Sufficient Stock,2024-07-19T05:29:39.939Z
2,Product 2,Garden,GardenMaster,497.76,495,3.8,true,Standard,Sufficient Stock,2024-07-19T05:29:39.939Z
3,Product 3,Electronics,BeautyGlow,331.63,10,4.6,true,Standard,Moderate Stock,2024-07-19T05:29:39.939Z
4,Product 4,Beauty,TechPro,798.83,683,4.7,false,Standard,Sufficient Stock,2024-07-19T05:29:39.939Z
5,Product 5,Automotive,HomeSmart,0.0,719,4.4,false,Budget,Sufficient Stock,2024-07-19T05:29:39.939Z
6,Product 6,Electronics,BookWorm,645.3,823,2.2,false,Standard,Sufficient Stock,2024-07-19T05:29:39.939Z
7,Product 7,Automotive,FashionX,549.08,999,1.1,false,Standard,Sufficient Stock,2024-07-19T05:29:39.939Z
8,Product 8,Books,TechPro,982.36,542,2.4,false,Standard,Sufficient Stock,2024-07-19T05:29:39.939Z
9,Product 9,Toys,FashionX,307.14,671,1.0,true,Standard,Sufficient Stock,2024-07-19T05:29:39.939Z
10,Product 10,Garden,BeautyGlow,871.38,975,3.4,false,Standard,Sufficient Stock,2024-07-19T05:29:39.939Z
